In [1]:
print("narender")

narender


In [2]:
import numpy 

In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from huggingface_hub import hf_hub_download
#from llama_cpp import Llama
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers   # for quantized model, we use ctransformers

d:\Generative_AI\chatbot_development\medical_chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
PINECONE_API_KEY = "enter pinecone API"

PINECONE_API_ENV = "gcp-starter"

## #Extract data from the PDF

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("D:\Generative_AI\data")

In [7]:
extracted_data[8]

Document(page_content=" \n \n9 Corta na  \nAs mentioned above, Cortana is example of a voice controlled intelligent system. It uses the \nBing search engine to carry out searches for the user and uses the data stored on the user's \ndevice to make personalized recommendations. Cortana is fully integrated into Windows 10 \nallowing it not only to search the Internet but via integration with the rest of the operating \nsystem can remind you of Calendar events, search for files on your device, open applications, \nand many other automated functions. Cort ana is also integrated into the Xbox allowing you to \nvoice control the entire system.  \n \nFig 1.3: Cortana - Personal Digital Assistant  \n  \n", metadata={'source': 'D:\\Generative_AI\\data\\AI_fundamentals.pdf', 'page': 8})

In [12]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)

print("length of my chunks:", len(text_chunks))

length of my chunks: 830


In [14]:
text_chunks

[Document(page_content='Fundamentals of  \nArtificial Intelligence  \nModule 1', metadata={'source': 'D:\\Generative_AI\\data\\AI_fundamentals.pdf', 'page': 0}),
 Document(page_content='2 Table of Contents', metadata={'source': 'D:\\Generative_AI\\data\\AI_fundamentals.pdf', 'page': 1}),
 Document(page_content='Introduction  ................................ ................................', metadata={'source': 'D:\\Generative_AI\\data\\AI_fundamentals.pdf', 'page': 1}),
 Document(page_content='................................ ................................ ....................  4', metadata={'source': 'D:\\Generative_AI\\data\\AI_fundamentals.pdf', 'page': 1}),
 Document(page_content='Learning Outcomes  ................................ ................................', metadata={'source': 'D:\\Generative_AI\\data\\AI_fundamentals.pdf', 'page': 1}),
 Document(page_content='................................ ................................ ....... 6', metadata={'source': 'D:\\Generati

In [15]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [18]:
query_result = embeddings.embed_query("Hello world")

print("Length", len(query_result))

Length 384


In [19]:
# vector representation

query_result

[-0.034477267414331436,
 0.031023206189274788,
 0.006734929047524929,
 0.026108982041478157,
 -0.03936203196644783,
 -0.16030243039131165,
 0.06692398339509964,
 -0.006441489793360233,
 -0.04745049402117729,
 0.014758839271962643,
 0.07087527960538864,
 0.05552763119339943,
 0.01919335499405861,
 -0.026251323521137238,
 -0.010109569877386093,
 -0.02694045566022396,
 0.022307392209768295,
 -0.02222662791609764,
 -0.14969263970851898,
 -0.017493031919002533,
 0.007676294539123774,
 0.054352253675460815,
 0.0032544503919780254,
 0.031725917011499405,
 -0.08462144434452057,
 -0.029405998066067696,
 0.051595620810985565,
 0.048124048858881,
 -0.00331486901268363,
 -0.058279212564229965,
 0.041969265788793564,
 0.022210726514458656,
 0.12818878889083862,
 -0.02233896590769291,
 -0.011656217277050018,
 0.06292834132909775,
 -0.032876402139663696,
 -0.09122605621814728,
 -0.03117532841861248,
 0.05269954353570938,
 0.047034818679094315,
 -0.08420311659574509,
 -0.030056139454245567,
 -0.020744

In [ ]:
#pip install -q --upgrade pinecone-client==2.2.4

In [20]:
#Initializing the Pinecone

pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name = "ai-fundamentals"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


query = "what is an AI?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result []


### If we already have an index in pinecone, so we can directly load it like this

In [21]:
#If we already have an index in pinecone, so we can directly load it like this
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name = "ai-fundamentals"
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "what is an AI?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='• Video: What is Artificial Intelligence (AI)?'), Document(page_content='Artificial Intelligence is an attempt to make a computer, a robot, or other piece of technology'), Document(page_content='Overview of AI  ................................ ................................')]


In [22]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [23]:
PROMPT=PromptTemplate(template=prompt_template, 
                      input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

In [4]:
# model_name_or_path = "TheBloke/Llama-2-7B-chat-GGML"

# model_basename = "llama-2-7b-chat.ggmlv3.q2_K.bin" # --> this model is small size 3gb.
# # model_basename = "llama-2-7b-chat.ggmlv3.q5_1.bin"   # -->  this model is large size 9gb.

In [5]:
# model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# model_path

llama-2-7b-chat.ggmlv3.q2_K.bin: 100%|██████████| 2.87G/2.87G [11:24<00:00, 4.19MB/s]
d:\Generative_AI\chatbot_development\medical_chatbot\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--TheBloke--Llama-2-7B-chat-GGML. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

'C:\\Users\\hp\\.cache\\huggingface\\hub\\models--TheBloke--Llama-2-7B-chat-GGML\\snapshots\\76cd63c351ae389e1d4b91cab2cf470aab11864b\\llama-2-7b-chat.ggmlv3.q2_K.bin'

In [24]:
llm=CTransformers(model = "D:\Generative_AI\llama-2-7b-chat.ggmlv3.q2_K.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8}) 

In [25]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [26]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

d:\Generative_AI\chatbot_development\medical_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Artificial intelligence (AI) refers to the simulation or creation of intelligent systems. This can include things like computer programs, software, and even robots. The goal of AI is to create machines that can think, learn, and make decisions like humans do.


In [38]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])